# 0. 필요 데이터 설치

In [8]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. 기상청 API 데이터 수집

In [ ]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 12, 31)
date_list = [(start_date + timedelta(days=i)).strftime('%Y%m%d') for i in range((end_date - start_date).days + 1)]

weather_records = []
col_names = [f'col{i}' for i in range(56)]
url_weather = "https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php"

for date in date_list:
    params_weather = {
        "tm1": date,
        "tm2": date,
        "stn": "108",
        "help": "1",
        "authKey": "n7wd-Im8T368HfiJvO9-5Q"  # 실제 인증키로 교체 필요
    }
    try:
        response_weather = requests.get(url_weather, params=params_weather, timeout=10)
        response_weather.encoding = 'euc-kr'
        lines = response_weather.text.split('\n')
        lines = [line for line in lines if line and not line.startswith('#')]
        if not lines:
            continue
        csv_data = '\n'.join(lines)
        df_w = pd.read_csv(StringIO(csv_data), sep=r'\s+', header=None, names=col_names, engine='python')
        df_w = df_w.reset_index(drop=True)
        df_w['date'] = df_w['col0'].astype(str).str[:8]
        df_w['date_dt'] = pd.to_datetime(df_w['date'], format='%Y%m%d', errors='coerce')
        df_w = df_w[['date_dt', 'col16', 'col21', 'col2']]
        df_w.columns = ['date_dt', 'TA_AVG', 'HM_AVG', 'WS_AVG']
        weather_records.append(df_w)
    except Exception as e:
        continue

if weather_records:
    df_weather_all = pd.concat(weather_records, ignore_index=True)
else:
    df_weather_all = pd.DataFrame()

print('기상청 데이터 수집 완료, 샘플 데이터:')
print(df_weather_all.head())
print('총 수집된 날짜 수:', df_weather_all['date_dt'].nunique())

기상청 데이터 수집 완료, 샘플 데이터:
     date_dt  TA_AVG  HM_AVG  WS_AVG
0 2022-01-01    -3.7     2.1     1.5
1 2022-01-02    -0.9     3.3     2.3
2 2022-01-03    -2.3     3.2     1.8
3 2022-01-04    -1.8     2.6     2.4
4 2022-01-05    -2.8     2.4     1.7
총 수집된 날짜 수: 1066


# 2. 산불위험예보 목록정보 전처리

In [9]:
df_risk = pd.read_csv('산림청 국립산림과학원_대형산불위험예보목록정보_20250430.csv', encoding='euc-kr')
df_risk = df_risk.rename(columns={
    '예보일시': 'date',
    '시도명': 'province',
    '시군구명': 'city',
    '실효습도': 'effective_humidity',
    '풍속': 'wind_speed',
    '등급': 'risk_grade'
})
df_risk['date_dt'] = pd.to_datetime(df_risk['date'], errors='coerce').dt.date
df_risk['effective_humidity'] = pd.to_numeric(df_risk['effective_humidity'], errors='coerce')
df_risk['wind_speed'] = pd.to_numeric(df_risk['wind_speed'], errors='coerce')

# 2022~2024년 데이터만 필터링
df_risk = df_risk[
    (df_risk['date_dt'] >= pd.to_datetime('2022-01-01').date()) &
    (df_risk['date_dt'] <= pd.to_datetime('2024-12-31').date())
]

In [10]:
df_fire = pd.read_csv('sanbul.csv', encoding='cp949')
df_fire = df_fire.rename(columns={
    '발생일시_년': 'year',
    '발생일시_월': 'month',
    '발생일시_일': 'day',
    '발생장소_시도': 'province',
    '발생장소_시군구': 'city'
})
df_fire['date_dt'] = pd.to_datetime(
    df_fire['year'].astype(str) + '-' +
    df_fire['month'].astype(str).str.zfill(2) + '-' +
    df_fire['day'].astype(str).str.zfill(2),
    errors='coerce'
).dt.date

# 2022~2024년 데이터만 필터링
df_fire = df_fire[
    (df_fire['date_dt'] >= pd.to_datetime('2022-01-01').date()) &
    (df_fire['date_dt'] <= pd.to_datetime('2024-12-31').date())
]

# 일별, 지역별 산불 발생 건수 및 유무 집계
fire_group = df_fire.groupby(['date_dt', 'province', 'city']).size().reset_index(name='fire_count')
fire_group['fire_occurred'] = (fire_group['fire_count'] > 0).astype(int)

In [11]:
# 1차 병합 (날짜 기준)
df_weather_all['date_dt'] = pd.to_datetime(df_weather_all['date_dt']).dt.date
risk_weather = pd.merge(df_risk, df_weather_all, on='date_dt', how='inner')

print('1차 병합(기상+위험예보) 샘플:', risk_weather.head())

# 2차 병합 (날짜+지역 기준)
final_df = pd.merge(
    risk_weather,
    fire_group,
    on=['date_dt', 'province', 'city'],
    how='left'
)

# 결측치 처리: 산불이 발생하지 않은 경우 0으로 채움
final_df['fire_occurred'] = final_df['fire_occurred'].fillna(0).astype(int)
final_df['fire_count'] = final_df['fire_count'].fillna(0).astype(int)

# 피처 엔지니어링: 위험 등급 수치화
final_df['risk_grade_num'] = final_df['risk_grade'].map({'주의보': 2, '경보': 3, '기타': 1})
features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed', 'risk_grade_num']

# feature 결측치 제거
final_df = final_df.dropna(subset=features)

print('타겟 변수 분포:', final_df['fire_occurred'].value_counts())

1차 병합(기상+위험예보) 샘플:                date province city 읍면동명  effective_humidity  wind_speed  \
0  2022-01-01 10:00       강원   양양  손양면                30.5         9.5   
1  2022-01-01 11:00       강원   양양  손양면                30.1         8.3   
2  2022-01-01 11:00       강원   양양  현북면                30.1         8.3   
3  2022-01-01 11:00       강원   양양  현남면                30.1         7.2   
4  2022-01-01 12:00       강원   양양  손양면                29.7         8.2   

  risk_grade     date_dt  TA_AVG  HM_AVG  WS_AVG  
0        주의보  2022-01-01    -3.7     2.1     1.5  
1        주의보  2022-01-01    -3.7     2.1     1.5  
2        주의보  2022-01-01    -3.7     2.1     1.5  
3        주의보  2022-01-01    -3.7     2.1     1.5  
4        주의보  2022-01-01    -3.7     2.1     1.5  
타겟 변수 분포: fire_occurred
0    13462
1      795
Name: count, dtype: int64


In [12]:
# SMOTE 적용 (0과 1이 모두 있을 때만 실행)
X = final_df[features].astype(float)
y = final_df['fire_occurred']

minority_count = sum(y == 1)

if minority_count < 2:
    print("소수 클래스 샘플 부족으로 SMOTE 적용 불가. 원본 데이터 사용")
    X_resampled, y_resampled = X, y
else:
    # 소수 클래스 샘플 수에 맞춰 k_neighbors 조정
    if minority_count < 6:
        smote = SMOTE(random_state=42, k_neighbors=minority_count-1)
    else:
        smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled,
    test_size=0.2,
    random_state=42,
    stratify=y_resampled
)

In [13]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'scale_pos_weight': [1, (y_resampled == 0).sum() / (y_resampled == 1).sum()]
}
model = XGBClassifier(random_state=42)
grid = GridSearchCV(model, param_grid, scoring='recall', cv=5)
grid.fit(X_train, y_train)
print("최적 파라미터:", grid.best_params_)

# 최적 모델로 예측
best_model = grid.best_estimator_
probs = best_model.predict_proba(X_test)[:, 1]

# 임계값 조정 (재현율 높이기)
threshold = 0.3
preds = (probs > threshold).astype(int)

print("\n임계값 조정 후 분류 리포트:")
print(classification_report(y_test, preds))
print("\n임계값 조정 후 혼동 행렬:")
print(confusion_matrix(y_test, preds))

최적 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'scale_pos_weight': 1}

임계값 조정 후 분류 리포트:
              precision    recall  f1-score   support

           0       0.99      0.87      0.93      2693
           1       0.89      0.99      0.94      2692

    accuracy                           0.93      5385
   macro avg       0.94      0.93      0.93      5385
weighted avg       0.94      0.93      0.93      5385


임계값 조정 후 혼동 행렬:
[[2350  343]
 [  23 2669]]


In [14]:
# 예측 결과 표로 확인
result_df = pd.concat([
    X_test.reset_index(drop=True),
    pd.DataFrame({
        'actual_fire_occurred': y_test,
        'predicted_fire_occurred': preds
    })
], axis=1)

print("\n예측 결과 샘플:")
print(result_df.head())
print("\n예측 결과 요약:")
print(result_df['predicted_fire_occurred'].value_counts())

# 모델 저장 (XGBoost 네이티브 방식)
best_model.save_model('xgb_fire_model_smote.json')

# 저장 확인
import os
print('모델 저장 여부:', os.path.exists('xgb_fire_model_smote.json'))


예측 결과 샘플:
   TA_AVG  HM_AVG  WS_AVG  effective_humidity  wind_speed  risk_grade_num  \
0    20.1     9.1     2.0           34.815684    7.200000             2.0   
1     4.4     2.7     4.3           38.171278   10.057233             2.0   
2    11.0    11.1     3.6           42.342022    7.786571             2.0   
3    -3.1     1.8     3.1           32.500000    7.885320             2.0   
4     7.2     3.8     3.4           35.200000    8.700000             2.0   

   actual_fire_occurred  predicted_fire_occurred  
0                   NaN                      NaN  
1                   0.0                      0.0  
2                   NaN                      NaN  
3                   NaN                      NaN  
4                   0.0                      0.0  

예측 결과 요약:
predicted_fire_occurred
1.0    3012
0.0    2373
Name: count, dtype: int64
모델 저장 여부: True
